In [2]:
#import bibliotheques python
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib

In [3]:
df = joblib.load('cleaned_data.sav')

In [4]:
df

index       Id                Tags  \
0          0        4                [c#]   
1          1        9          [c#, .net]   
2          2       11                [c#]   
3          3       16                [c#]   
4          4       17   [mysql, database]   
...      ...      ...                 ...   
41163  41163  8057565             [mysql]   
41164  41164  8054165              [html]   
41165  41165  8064691  [javascript, html]   
41166  41166  8057607              [java]   
41167  41167  8067590        [javascript]   

                                                    Body  java   c#  \
0      convert decim doubl track bar form opac lang p...   0.0  1.0   
1      calcul someon age base datetim birthday given ...   0.0  1.0   
2      calcul relat given specif datetim display rela...   0.0  1.0   
3      fill dataset datat linq queri result expo linq...   0.0  1.0   
4      binari mysql store binari http en wikipedia wi...   0.0  0.0   
...                                                  ...   ...  ...   
41163  mysql updat inner join queri idea problem mysq...   0.0  0.0   
41164                        put form put form send form   0.0  0.0   
41165  pas along variabl xmlhttprequest send variabl ...   0.0  0.0   
41166  mock privat powermock still mock privat jndi u...   1.0  0.0   
41167  associ versus hash search googl folk recommend...   0.0  0.0   

       javascript  python  jquery  html  ...  regex  iphone  bash  \
0             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
1             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
2             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
3             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
4             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
...           ...     ...     ...   ...  ...    ...     ...   ...   
41163         0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
41164         0.0     0.0     0.0   1.0  ...    0.0     0.0   0.0   
41165         1.0     0.0     0.0   1.0  ...    0.0     0.0   0.0   
41166         0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
41167         1.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   

       objective-c  ios  windows  ruby  eclipse  tsql  database  
0              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
1              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
2              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
3              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
4              0.0  0.0      0.0   0.0      0.0   0.0       1.0  
...            ...  ...      ...   ...      ...   ...       ...  
41163          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41164          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41165          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41166          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41167          0.0  0.0      0.0   0.0      0.0   0.0       0.0  

[41168 rows x 34 columns]

# Training data

In [6]:
X = df.Body
Y = df.Tags

# Unsupervided Method

## LDA

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_fited = tfidf_vec.fit_transform(X)

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10,max_iter=5,learning_method='online',learning_offset=50,random_state = 0).fit(X_fited)

In [8]:
 def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 1
display_topics(lda, tfidf_vec.get_feature_names(), no_top_words)

Topic 0:
string
Topic 1:
http
Topic 2:
date
Topic 3:
button
Topic 4:
command
Topic 5:
git
Topic 6:
list
Topic 7:
android
Topic 8:
sql
Topic 9:
java


## NMF

In [9]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
nmf.fit(X_fited)

NMF(alpha=0.1, init='nndsvd', l1_ratio=0.5, n_components=10, random_state=1)

In [10]:
no_top_words = 1
display_topics(nmf, tfidf_vec.get_feature_names(), no_top_words)

Topic 0:
http
Topic 1:
string
Topic 2:
android
Topic 3:
java
Topic 4:
list
Topic 5:
sql
Topic 6:
git
Topic 7:
date
Topic 8:
button
Topic 9:
cs


# Supervised Method

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [29]:
cat_df = df.drop(columns = ['Id','index','Body','Tags'])
#cat = cat_df.columns

In [23]:
#fitting_dict = {}
#fitting_list = []
#for i in range(len(cat)):
    #fitting_dict.update({cat[i]:i})
    #fitting_list.append([cat[i],i])

In [24]:
Y_fitted = mlb.fit_transform(Y)

## Train/Test Split

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_fitted, test_size=0.33, random_state=42)

## TF-IDF

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_train = tfidf_vec.fit_transform(X_train)
X_test = tfidf_vec.transform(X_test)

## Scores

In [30]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# liste des 10 catégories
categories = list(cat_df.columns.values)
# liste des métrics utilisé
scores = ['f1','accuracy','precision','recall']
# liste des modèles utilisé
modeles = ['Logistic_Regression','SVM','Naive_Bayes','MLP']
# Tableaux qui vont acceuillir les score pour de chaque model pour l'approche One_vs_rest
Logistic_Regression = pd.DataFrame(columns = scores,index = categories)
SVM = pd.DataFrame(columns = scores,index = categories)
Naive_Bayes = pd.DataFrame(columns = scores,index = categories)
MLP = pd.DataFrame(columns = scores,index = categories)
#Tableau qui va acceuillir les scores pour l'approche Binary Relevance
Binary_relevance_score = pd.DataFrame(index=scores,columns=modeles)
#Tableau qui va acceuillir les scores pour l'approche LabelPowerset
Label_powerset_score = pd.DataFrame(index=scores,columns=modeles)
#Tableau qui va acceuillir les scores pour le model MLKNN
MLKNN_Score = pd.DataFrame(index=['MLKNN'],columns=scores)

## MLKNN

In [31]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train).toarray()
Y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test).toarray()
#parameter
parameters = {'k': range(1,6)}
# train
clf = GridSearchCV(MLkNN(), parameters, scoring='f1_macro')
clf.fit(X_train, Y_train)
#best param
print (clf.best_params_, clf.best_score_)
# predict
predictions_new = clf.predict(x_test)

/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarni

{'k': 4} 0.3458601189498148


In [32]:

MLKNN_accuracy = accuracy_score(y_test, predictions_new)
MLKNN_precision = precision_score(y_test, predictions_new,average='weighted')
MLKNN_recall = recall_score(y_test, predictions_new,average='weighted')

MLKNN_Score['accuracy']=MLKNN_accuracy
MLKNN_Score['precision']=MLKNN_precision
MLKNN_Score['recall']=MLKNN_recall

## Logistique Regression

### Binary Relevance

In [34]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(LogisticRegression(penalty = 'l2' ,max_iter = 200,solver='sag'))
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [35]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['Logistic_Regression']=Score


### Label Powerset

In [36]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression(penalty = 'l2' ,max_iter = 200,solver='sag'))
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [37]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['Logistic_Regression']=Score


## Support vector Classifier

### Label Powerset

In [38]:
# using Label Powerset
from sklearn.svm import SVC
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(SVC())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [39]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['SVM']=Score


### Binary Relevance

In [40]:
# using binary relevance 
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(SVC())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [41]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['SVM']=Score


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MultinomialNB

### Binary Relevance

In [44]:
from sklearn.naive_bayes import MultinomialNB
# using binary relevance 
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [45]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['Naive_Bayes']=Score


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Label Powerset

In [46]:
# using Label Powerset
from sklearn.svm import SVC
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [47]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['Naive_Bayes']=Score


/home/abdifatah/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MLP Classifier

### Binary Relevance

In [48]:
# using binary relevance 
from skmultilearn.problem_transform import BinaryRelevance
# initialize binary relevance multi-label classifier
classifier = BinaryRelevance(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [ ]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Binary_relevance_score['MLP']=Score

### Label Powerset

In [ ]:
# using Label Powerset
from sklearn.svm import SVC
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(MultinomialNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [ ]:
Score = []


Score.append(f1_score(y_test, predictions,average='weighted'))
Score.append(accuracy_score(y_test, predictions))
Score.append(precision_score(y_test, predictions,average='weighted'))
Score.append(recall_score(y_test, predictions,average='weighted'))

Label_powerset_score['MLP']=Score

# Scores display

In [ ]:
MLKNN_Score 

In [ ]:
def average_scores(series):
    average_scores = pd.DataFrame()
    average_f1_score = []
    average_accuracy = []
    average_precision = []
    average_recall = []
    average_f1_score.append(sum(series.f1)/len(series.f1))
    average_accuracy.append(sum(series.accuracy)/len(series.accuracy))
    average_precision.append(sum(series.precision)/len(series.precision))
    average_recall.append(sum(series.recall)/len(series.recall))
    average_scores['f1_score'] = average_f1_score
    average_scores['accuracy'] = average_accuracy
    average_scores['precision'] = average_precision
    average_scores['recall'] = average_recall
    return average_scores

In [40]:
average_log_reg = average_scores(Logistic_Regression)
average_svc = average_scores(SVM)
average_MNB = average_scores(Naive_Bayes)
average_MLP = average_scores(MLP)

In [41]:
average_log_reg

f1_score  accuracy  precision    recall
0  0.449659  0.967565   0.719561  0.354251

In [42]:
average_svc

f1_score  accuracy  precision    recall
0  0.434268   0.96846   0.745449  0.352039

In [43]:
average_MNB

f1_score  accuracy  precision    recall
0  0.240389  0.961487   0.741924  0.169793

In [44]:
average_MLP

f1_score  accuracy  precision    recall
0  0.000345    0.9549   0.033333  0.000174

In [45]:
Binary_relevance_score

Logistic_Regression       SVM  Naive_Bayes       MLP
f1                    0.475893  0.464338     0.263908  0.263908
accuracy              0.295083  0.309804     0.151995  0.151995
precision             0.747386  0.771194     0.798453  0.798453
recall                0.371581  0.365969     0.181323  0.181323

In [46]:
Label_powerset_score

Logistic_Regression       SVM  Naive_Bayes       MLP
f1                    0.525896  0.519203     0.434288  0.434288
accuracy              0.432357  0.430958     0.388415  0.388415
precision             0.611735  0.617244     0.594565  0.594565
recall                0.496949  0.488395     0.429116  0.429116

# Best model

In [52]:
Logistic_Regression

f1  accuracy  precision    recall
java         0.712920  0.931474   0.903831  0.588595
c#           0.392554  0.889519   0.712188  0.270950
javascript   0.424053  0.904828   0.693878  0.305324
python       0.498117  0.931326   0.849541  0.352359
jquery       0.460073  0.945753   0.744076  0.332980
html         0.338188  0.939791   0.554377  0.243306
php          0.193611  0.938687   0.699301  0.112360
android      0.859569  0.984175   0.983558  0.763341
.net         0.065445  0.947446   0.595238  0.034626
c++          0.412460  0.959738   0.764940  0.282353
sql          0.503145  0.959296   0.632054  0.417910
css          0.746492  0.980053   0.865510  0.656250
mysql        0.805430  0.987340   0.912821  0.720648
string       0.422572  0.967614   0.589744  0.329243
sql-server   0.558940  0.975489   0.730104  0.452790
c            0.257143  0.973208   0.768293  0.154412
git          0.943089  0.996909   0.997135  0.894602
arrays       0.005666  0.974165   0.125000  0.002899
asp.net      0.475294  0.983586   0.827869  0.333333
linux        0.426150  0.982556   0.807339  0.289474
regex        0.751620  0.991535   0.930481  0.630435
iphone       0.417391  0.985205   0.774194  0.285714
bash         0.671498  0.989990   0.908497  0.532567
objective-c  0.022814  0.981083   0.214286  0.012048
ios          0.332248  0.984911   0.680000  0.219828
windows      0.051724  0.983807   0.666667  0.026906
ruby         0.655914  0.990579   0.960630  0.497959
eclipse      0.809524  0.995289   0.931507  0.715789
tsql         0.131455  0.986383   0.368421  0.080000
database     0.144681  0.985205   0.395349  0.088542

In [57]:
SVM

f1  accuracy  precision    recall  f1_score
java         NaN  0.937804   0.920993  0.623218  0.743395
c#           NaN  0.890917   0.809237  0.225140  0.352273
javascript   NaN  0.907478   0.738170  0.300192  0.426813
python       NaN  0.931768   0.902287  0.330289  0.483565
jquery       NaN  0.946636   0.811429  0.301166  0.439288
html         NaN  0.941631   0.621324  0.196740  0.298851
php          NaN  0.938760   0.750000  0.097753  0.172962
android      NaN  0.987193   0.980447  0.814385  0.889734
.net         NaN  0.946857   0.000000  0.000000  0.000000
c++          NaN  0.960548   0.923529  0.230882  0.369412
sql          NaN  0.961431   0.657328  0.455224  0.537919
css          NaN  0.981451   0.861789  0.697368  0.770909
mysql        NaN  0.987855   0.885246  0.765182  0.820847
string       NaN  0.967540   0.636364  0.229039  0.336842
sql-server   NaN  0.975858   0.731544  0.467811  0.570681
c            NaN  0.971147   0.863636  0.046569  0.088372
git          NaN  0.998013   0.997253  0.933162  0.964143
arrays       NaN  0.974606   0.500000  0.002899  0.005764
asp.net      NaN  0.983954   0.781457  0.389439  0.519824
linux        NaN  0.982776   0.777778  0.322368  0.455814
regex        NaN  0.992198   0.916667  0.677536  0.779167
iphone       NaN  0.985279   0.754902  0.305556  0.435028
bash         NaN  0.991094   0.916667  0.590038  0.717949
objective-c  NaN  0.981599   0.000000  0.000000  0.000000
ios          NaN  0.984690   0.800000  0.137931  0.235294
windows      NaN  0.983660   1.000000  0.004484  0.008929
ruby         NaN  0.991977   0.935897  0.595918  0.728180
eclipse      NaN  0.995878   0.889535  0.805263  0.845304
tsql         NaN  0.987119   0.000000  0.000000  0.000000
database     NaN  0.986089   1.000000  0.015625  0.030769